# Ejercicio 4

El conjunto de datos binary.csv contiene información de la admisión de estudiantes a una
universidad. Las variables son:  

admit: (toma valores 0: no fue admitido, 1 fue admitido),  
GRE: (Graduate Record Exam scores) variable numérica,  
GP A: (grade point average) variable numérica,  
rank: variable categórica que se refiere al prestigio de la escuela secundaria a la que
el alumno asistió y toma valores {1, 2, 3, 4}.  
Un investigador está interesado en averiguar cómo influyen estas variables en la admisión.
Discretiza las variables GRE y GP A de la siguiente manera GRE ∈ {GRE ≥
500, GRE < 500} y GP A ∈ {GP A ≥ 3, GP A < 3}. Sabe que estas variables cumplen las
relaciones presentadas en la Figura 1.  

<img src="figura1.png">

a) Calcular la probabilidad de que una persona que proviene de una escuela con rango
1 no haya sido admitida en la universidad.  
b) Calcular la probabilidad de que una persona que fue a una escuela de rango 2, tenga
GRE = 450 y GPA = 3.5 sea admitida en la universidad.  
c) En este ejercicio, ¿cuál es el proceso de aprendizaje?

In [6]:
%load_ext autoreload
%autoreload 2
import pandas as pd
df = pd.read_csv("binary.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
df.head()

# Adecuación de las variables de entrada

Voy a convertir las variables de entrada de acuerdo a la umbralización definida en el punto 1. De esta forma queda:

admin: distribución de Bernoulli  
gre: distribución de Bernoulli  
gpa: distribución de Bernoulli  
rank: distribución categórica 

In [8]:
df["gre"] = 1*(df["gre"]>=500)
df["gpa"] = 1*(df["gpa"]>=3)

In [9]:
df.head()

,admit,gre,gpa,rank
0,0,0,1,3
1,1,1,1,3
2,1,1,1,1
3,1,1,1,4
4,0,1,0,4


# Factorización de la conjunta

Mirando el grafo podemos plantear la probabilidad conjunta en función de las siguientes independencias condicionales:


$P(admit,gre,gpa,rank)= P(admit|gre,gpa,rank)P(gre,gpa,rank)$  
$P(admit,gre,gpa,rank)= P(admit|gre,gpa,rank)P(gre,gpa|rank)P(rank)$  
$P(admit,gre,gpa,rank)= P(admit|gre,gpa,rank)P(gre|rank)P(gpa|rank)P(rank)$  

Ahora comienza nuestro proceso de aprendizaje, que es estimar estas probabilidades a partir de los datos. Podemos empezar por la mas sencilla que es P(rank):

In [22]:
rank_values = list(df["rank"].unique())
gpa_values = list(df["gpa"].unique())
gre_values = list(df["gre"].unique())
admit_values = list(df["admit"].unique())

In [23]:
# Me fijo cuántas veces aparece cada valor de rank y normalizo
p_rank = dict(df["rank"].value_counts()/len(df))

In [24]:
p_rank

{2: 0.3775, 3: 0.3025, 4: 0.1675, 1: 0.1525}

In [40]:
p_gpa = {rank_value:dict(df[df["rank"]==rank_value]["gpa"].value_counts()/len(df[df["rank"]==rank_value])) for rank_value in rank_values}

In [41]:
p_gre = {rank_value:dict(df[df["rank"]==rank_value]["gre"].value_counts()/len(df[df["rank"]==rank_value])) for rank_value in rank_values}

In [44]:
p_gre

{3: {1: 0.7933884297520661, 0: 0.2066115702479339},
 1: {1: 0.819672131147541, 0: 0.18032786885245902},
 4: {1: 0.7910447761194029, 0: 0.208955223880597},
 2: {1: 0.8145695364238411, 0: 0.18543046357615894}}

In [45]:
p_gpa

{3: {1: 0.8347107438016529, 0: 0.1652892561983471},
 1: {1: 0.8688524590163934, 0: 0.13114754098360656},
 4: {1: 0.8059701492537313, 0: 0.19402985074626866},
 2: {1: 0.8278145695364238, 0: 0.17218543046357615}}

In [55]:
p_admit = {}
for rank_value in rank_values:
    for gpa_value in gpa_values:
        for gre_value in gre_values:
            datos = df[(df["rank"] ==rank_value) & (df["gpa"] == gpa_value) & (df["gre"] == gre_value)]
            p_admit[(rank_value,gpa_value,gre_value)] = dict(datos["admit"].value_counts()/len(datos))

In [56]:
p_admit[(3,1,0)]

{0: 0.8125, 1: 0.1875}

In [57]:
len(p_admit)

16

In [58]:
p_admit

{(3, 1, 0): {0: 0.8125, 1: 0.1875},
 (3, 1, 1): {0: 0.7529411764705882, 1: 0.24705882352941178},
 (3, 0, 0): {0: 1.0},
 (3, 0, 1): {0: 0.6363636363636364, 1: 0.36363636363636365},
 (1, 1, 0): {1: 0.5, 0: 0.5},
 (1, 1, 1): {1: 0.5531914893617021, 0: 0.44680851063829785},
 (1, 0, 0): {0: 0.8, 1: 0.2},
 (1, 0, 1): {1: 1.0},
 (4, 1, 0): {0: 0.8, 1: 0.2},
 (4, 1, 1): {0: 0.7954545454545454, 1: 0.20454545454545456},
 (4, 0, 0): {0: 1.0},
 (4, 0, 1): {0: 0.8888888888888888, 1: 0.1111111111111111},
 (2, 1, 0): {0: 0.8095238095238095, 1: 0.19047619047619047},
 (2, 1, 1): {0: 0.5769230769230769, 1: 0.4230769230769231},
 (2, 0, 0): {0: 0.5714285714285714, 1: 0.42857142857142855},
 (2, 0, 1): {0: 0.8421052631578947, 1: 0.15789473684210525}}

## a) Calcular la probabilidad de que una persona que proviene de una escuela con rango 1 no haya sido admitida en la universidad.

Se pide $P(admit=0|rango=1)$  

Tengo que fijar admit en 0 y rango en 1 y sumar con respecto a GPA y GRE para tener $P(admit, rank)$  
Tengo que fijar rango en 1 y sumar con respecto a GPA, GRE y admit para tener $P(rank)$

In [87]:
rank = 1
p_ar = 0
p_r = 0
for gpa in gpa_values:
    for gre in gre_values:
        p_ar += p_admit[(rank,gpa,gre)].get(0,0)*p_gpa[rank].get(gpa,0)*p_gre[rank].get(gre,0)*p_rank.get(rank,0)
for admit in admit_values:
    for gpa in gpa_values:
        for gre in gre_values:
            p_r += p_admit[(rank,gpa,gre)].get(admit,0)*p_gpa[rank].get(gpa,0)*p_gre[rank].get(gre,0)*p_rank.get(rank,0)

In [88]:
p_ar/p_r

0.41546427121512747

In [89]:
# Verificación rápida calculando los valores directo del dataset

In [90]:
df[df["rank"]==1]["admit"].value_counts()/len(df[df["rank"]==1])

1    0.540984
0    0.459016
Name: admit, dtype: float64

Interesante, no da lo mismo porque no estoy aplicando la condición de independencia condicional entre GRE y GPA.

## b) Calcular la probabilidad de que una persona que fue a una escuela de rango 2, tenga GRE = 450 y GPA = 3.5 sea admitida en la universidad. 

Eso implica rank=2, GRE=0, GPA=1.  
Se pide $P(admit=1|rank=2, GPA=1, GRE=0)$.  

In [91]:
p_admit[(2,1,0)]

{0: 0.8095238095238095, 1: 0.19047619047619047}

## c) En este ejercicio, ¿cuál es el proceso de aprendizaje?

El proceso de aprendizaje ocurrió cuando estimamos las probabilidades condicionales.